In [1]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.callbacks.manager import CallbackManager
from langchain.prompts.prompt import PromptTemplate
from langchain.vectorstores.base import VectorStoreRetriever
from langchain.vectorstores.faiss import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import ChatGLM
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationBufferWindowMemory
import pickle
from langchain.document_loaders import JSONLoader
import sys
import os
from dotenv import load_dotenv

In [2]:
db_path = "vector_src/mh_src-vectorstore.pkl"
with open(db_path, 'rb') as f:
    db = pickle.load(f)
# retriever = VectorStoreRetriever(vectorstore=db)
retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 8})


/home/dell/miniconda3/envs/gpt/lib/python3.11/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [136]:
docs = retriever.get_relevant_documents(query="电天气下民航飞机的飞行风险及预防措施分析中的国内外研究进展有哪些?")
content = "\n".join([doc.page_content for doc in docs])
print(content)
print(len(content))

相关关键词2023/10/29雷雨天气条件下保障航空飞行安全的有效措施-中国期刊网https:///thesis/view/47660611/3
民航安全一直是我国当局的工作重心，现代飞机航电系统交联关系复杂，系统高度集成，惯导系统又是高度精密的设备，其与飞机上的其他各项设备关联功能的完整性任需加强。
编号南京航空航天大学毕业论文题目气象条件对于民航飞行安全的影响学生姓名奚晓迪学号070850605学院民航飞行学院专业飞行技术班级0708506指导教师陆中教授二零一叁年伍月贰拾日
首页期刊导航论文中心期刊检索论文检索新闻中心恶劣天气对飞行安全的影响柴栋梁中国东方航空公司山西分公司山西省太原市030031摘要：恶劣天气对飞行安全的影响是巨大和致命性的，包括大风、云层、气压、气温、大气密度、雷电、冰雪等在内的恶劣天气均会对飞行安全产生极大的危害，严重时会引发空难。鉴于恶劣天气会导致飞行安全的控制工作遇到诸多的难题，因而一直以来航空公司均可以给予飞行安全高度的重视，这也是航空公司一直研究的重点问题。本文先对飞机安全飞行的重要性作简要分析，进而重点分析探究大风、云层、雷电、冰雪、雷雨、结冰对飞行安全的影响，最后提出降低恶劣天气对飞行安全的影响，确保飞行安全的策略。关键词：恶劣天气；飞行安全；气象环境天气是地球对动力、辐射、电能、水汽、气压的精巧平衡，之所以飞机飞行时格外注重天气因素，其原因就在于恶劣的天气会直接导致飞机处于极度危险的境地。每年因为恶劣天气所导致的民航事故可以占到总事故的四分之一到二分之一，每年因为恶劣天气所导致的航班延误和事故损失可以达到10亿美元以上。由此可以看出，恶劣天气对飞行
首页期刊导航论文中心期刊检索论文检索新闻中心浅谈雷雨天气下民航飞行特点张芸民航贵州空管分局摘要：众所周知，民航在飞行时会受到天气和民航动能等多方面因素的影响，无形中加大民航飞行时出现安全事故的可能。而雷雨天气作为民航飞行时经常遇到的天气状况，在这种天气状况条件下对流层中大气运行稳定性较差，造成民航飞行稳定性下降，这对于民航飞行安全效果也有很大的影响。本文将对雷雨天气综合分析，并根据分析结果概述雷雨天气下民航飞行特点。关键词：雷雨天气；民航飞行；特点引言与其他交通工具相比，民航飞行高度多半在对流层这一高度范围内。这就导致民航飞行时经常会受到恶劣天气的干扰，造成民航飞行安全性下降，难

In [159]:
from gc import set_debug
from langchain.chains import ConversationChain
from langchain.chains import LLMChain
from langchain.schema import StrOutputParser
from langchain.llms import OpenAI
from langchain.globals import set_verbose
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import ChatPromptTemplate
from pyparsing import quotedString
from query_data import get_llm
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# from langchain import LCEL
import logging
set_verbose(True)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_base="http://localhost:8000/v1",
    openai_api_key="EMPTY",
    max_tokens=8000,
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
# llm = ChatOpenAI(
#     openai_api_base="https://aiapi.xing-yun.cn/v1",
#     openai_api_key="sk-3e5wTBAl2iFDvQvW9b5693C90a97425eBf3b4bEa558eC66a",
#     temperature=0.1,
#     model_name="gpt-3.5-turbo",
#     streaming=True,  # ! important
#     callbacks=[StreamingStdOutCallbackHandler()]  # ! important
# )


# <step 1> 提取出所有的文章名/作者名/数据/时间等,你的回答必须包函这些数据
# <step 2> 根据你所知道的全部信息和相关的提问,整理出一大段有用的信息.
docs = retriever.get_relevant_documents(query="电天气下民航飞机的飞行风险及预防措施分析中的国内外研究进展有哪些?")
content = "\n".join([doc.page_content for doc in docs])
template_zh = """
### Instruction ###
您是一个很能写的写作助手,写的文章字数又多,文章又优美.能够根据下面<context>中的所有信息写出文章.
多引用背景材料, 对每一个例子都要有具体的分析, 避免出现总结性的语句.如果出现相关的数据,作者名,文章名,必须要全部引用出来 .不能漏掉. 
如果您不知道答案,只需说“嗯,我不确定”。不要试图编造答案。
写作风格符合常见的中文学术写作风格。要包函<context>中的所有内容.
<context>
背景: {context}
<context/>
用中文输出.1000字
"""
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers").setLevel(logging.INFO)
question = "雷电天气下民航飞机的飞行风险及预防措施分析中背景?"
# question = "科学与技术讲了什么"
# q = "在上述文本中提取出有用的信息"
# context = "在雷电天气下，民航飞机的飞行面临着一定的风险。为了确保飞行安全，需要对雷电天气下的飞行风险进行分析，并制定相应的预防措施。本研究旨在探讨雷电天气对民航飞机飞行的影响，并研究相关的飞行风险和预防措施。"

prompt = ChatPromptTemplate.from_template(template=template_zh)
# model = ConversationChain(llm=llm)
llm = get_llm('local')
template = """Answer the question based only on the following context:
<context>
{context}
</context>"""
ANSWER_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", template_zh),
        ("user", "{question}"),
    ]
)
from langchain.chains import RetrievalQA
chain_type_kwargs = {"prompt": ANSWER_PROMPT}
qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever, chain_type_kwargs=chain_type_kwargs)
chain = ANSWER_PROMPT | llm | StrOutputParser()
# chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | ANSWER_PROMPT
#     | llm 
#     | StrOutputParser()
# )
# output=chain.invoke(question)
output = chain.invoke({"context":content, "question": question})
# qa.run(question)
print(output)


 在分析雷电天气下民航飞机的飞行风险及预防措施时，我们需要先了解雷电天气对飞机飞行安全的影响。雷电天气是一种极端的气象现象，具有很高的危险性。当飞机处于雷电天气中时，可能会遭受闪电击、雷击、电磁干扰等多种影响，这些影响可能导致飞机设备损坏、飞行数据丢失、机组人员受伤甚至死亡等严重后果（张芸，2016）。因此，在雷电天气下，民航飞机的飞行风险非常高，需要采取一系列预防措施来降低风险（耿海周新辉，2022）。

在实际操作中，飞行员需要根据气象预报和实况，提前规划飞行路线，避开雷电天气区域。此外，飞行员还需要定期进行雷电天气应对培训，熟悉各种应对策略和紧急处理方法，以便在遇到突发情况时能够迅速做出反应（李浩，2020）。同时，航空公司也可以通过购买保险来分散风险，减轻因雷电天气引发的损失（李纯柱，2020）。

在预防和应对雷电天气的过程中，气象部门的作用同样重要。他们需要提高气象预报的准确性和时效性，为航空公司和飞行员提供及时、可靠的信息支持。此外，气象部门还应该加强对雷电天气的研究，探索更加有效的预警手段和防范措施，以保障民航飞行安全（郑洁，2020）。

综上所述，在雷电天气下，民航飞机的飞行风险较高。为了降低风险，航空公司需要采取一系列预防措施，包括提前规划飞行路线、加强飞行员培训、购买保险等。同时，气象部门也需要提高气象预报的准确性和时效性，加强研究雷电天气，为民航飞行提供有力保障。


In [153]:

question_generator_template = """
Take deep and think step by step.
现在你是一个提问小助手, 你非常善于提问,但不会回答问题.你的任务是根据下面提供的问题将问题进行扩展, 制造出更多的问题,用来指导写作.只提出5个问题就可以了.要从不同的角度去提问,
因为我想得到不同观点的相关信息.
=========
你输出的是一系列的问题而不是分析.要以问号结尾.参考如下问题:
Template:
<Question>: 什么叫废物固体处理?我们可以从废物固处理得到什么?
<Output>:
    1. 废物固体处理的定义是什么，它与其他废物处理方式有何不同？
    2. 废物固体处理的历史背景和发展是怎样的？
    3. 哪些因素促使我们需要对废物固体进行处理？
    4. 废物固体处理的各个阶段包括哪些步骤？
    5. 哪些技术和方法常用于废物固体处理？
=========
Question是: {question}
<Output>:
"""

question_generator = PromptTemplate.from_template(question_generator_template)
chain = question_generator | llm | StrOutputParser()
chain.invoke({"question":question})

'\n 1. 在雷电天气下，为什么民航飞机的飞行会面临风险？\n2. 有哪些具体的气象条件可能导致雷电天气的出现？\n3. 在雷电天气下，为民航飞机的飞行采取哪些预防措施可以降低风险？\n4. 对于已经发生的雷电天气导致的事故，有哪些教训和经验可供借鉴？\n5. 如何通过科技手段来监测和预警雷电天气，以提高民航飞机的安全性？'

In [175]:

question_generator_template = """
你的任务是把下面的陈述句改成一个有启发性的疑问句,只输出疑问句:
陈述句:{question}
疑问句:
"""
openai_llm = ChatOpenAI(
    openai_api_base="https://aiapi.xing-yun.cn/v1",
    openai_api_key="sk-3e5wTBAl2iFDvQvW9b5693C90a97425eBf3b4bEa558eC66a",
    temperature=0.0,
    model_name="gpt-4",
    # streaming=True,  # ! important
    # callbacks=[StreamingStdOutCallbackHandler()]  # ! important
)
question_generator = PromptTemplate(template=question_generator_template, input_variables=["question"])
# chain = question_generator | llm | StrOutputParser()
# chain.invoke({"question":question})
question = "雷电天气下民航飞机的飞行风险及预防措施分析中, 研究背景"
sentence_change_chain = LLMChain(prompt=question_generator, llm=openai_llm)
sentence_change_chain.run(question=question)



> Entering new LLMChain chain...
Prompt after formatting:

你的任务是把下面的陈述句改成一个有启发性的疑问句,只输出疑问句:
陈述句:雷电天气下民航飞机的飞行风险及预防措施分析中, 研究背景
疑问句:


> Finished chain.


'在分析雷电天气下民航飞机的飞行风险及预防措施的研究背景时，我们应该考虑哪些关键因素？'

In [51]:
get_llm('local')


ChatOpenAI(callbacks=[<langchain.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f210b6296d0>], client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, temperature=0.1, openai_api_key='EMPTY', openai_api_base='http://localhost:8000/v1', openai_organization='', openai_proxy='', streaming=True, max_tokens=8000)

In [ ]:
import os
from operator import itemgetter
from typing import List, Tuple
from langchain.globals import set_verbose
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, format_document
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import (
    RunnableBranch,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)
from pydantic import BaseModel, Field
set_verbose(True)
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

# RAG answer synthesis prompt
template = """Answer the question based only on the following context:
<context>
{context}
</context>"""
ANSWER_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{question}"),
    ]
)

# Conversational Retrieval Chain
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        print(f"human{human}, ai:{ai}")
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    print(buffer)
    return buffer


# User input
class ChatHistory(BaseModel):
    chat_history: List[Tuple[str, str]]
    question: str


_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(itemgetter("question")),
)

_inputs = RunnableMap(
    {
        "question": lambda x: x["question"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
        "context": _search_query | retriever | _combine_documents,
    }
).with_types(input_type=ChatHistory)
openai_llm = ChatOpenAI(
    openai_api_base="https://aiapi.xing-yun.cn/v1",
    openai_api_key="sk-3e5wTBAl2iFDvQvW9b5693C90a97425eBf3b4bEa558eC66a",
    temperature=0.1,
    model_name="gpt-4",
)
chain = _inputs | ANSWER_PROMPT | llm | StrOutputParser()

chat_history = [("hi", "hello")]
answer = chain.invoke(
    {
        "question":question,
        "chat_history":[("hi","AIMessage")]
    }
)

In [79]:
answer

'\n 在雷电天气下，民航飞机的飞行风险增加，这对民航飞行安全提出了挑战。根据提供的背景资料，我们可以了解到以下几点：\n\n1. 雷雨天气对飞机飞行的影响：雷雨天气会对民航飞行产生负面影响，如降低飞行稳定性、增大飞行误差、引发设备故障等，从而增加飞行安全风险。\n\n2. 民航飞行中雷电天气的安全事故比例较高：据统计，民航在飞行过程中因雷电天气而出现的安全事故在所有安全事故中占比较高，这进一步强调了加强雷电天气下民航飞行安全研究的必要性。\n\n3. 现有防雷技术及措施存在不足：针对雷电天气下民航飞机的飞行风险，目前采用的防雷技术和措施仍存在一定的局限性，需要不断研究和改进。\n\n因此，在雷电天气下民航飞机的飞行风险及预防措施分析中，我们需要对这些问题进行深入研究，以期提出有效的解决方案，保障民航飞行安全。'

In [80]:
print(answer)


 在雷电天气下，民航飞机的飞行风险增加，这对民航飞行安全提出了挑战。根据提供的背景资料，我们可以了解到以下几点：

1. 雷雨天气对飞机飞行的影响：雷雨天气会对民航飞行产生负面影响，如降低飞行稳定性、增大飞行误差、引发设备故障等，从而增加飞行安全风险。

2. 民航飞行中雷电天气的安全事故比例较高：据统计，民航在飞行过程中因雷电天气而出现的安全事故在所有安全事故中占比较高，这进一步强调了加强雷电天气下民航飞行安全研究的必要性。

3. 现有防雷技术及措施存在不足：针对雷电天气下民航飞机的飞行风险，目前采用的防雷技术和措施仍存在一定的局限性，需要不断研究和改进。

因此，在雷电天气下民航飞机的飞行风险及预防措施分析中，我们需要对这些问题进行深入研究，以期提出有效的解决方案，保障民航飞行安全。


In [50]:
from langchain.document_loaders import JSONLoader
from langchain.globals import set_verbose
set_verbose(True)
_template = """
<信息>{context}</信息>
根据上面的信息, 写出关于雷电天气下民航飞机的飞行风险及预防措施分析的研究目的, 只写一个段落, 作为论文的一部份.要具体的分析,有相关的作者,文献,数据就要提出来详细说明.
请用一种连贯的叙述方式来阐述你的观点和论据。请用丰富的词汇和多样的句式来表达你的观点,让你的回答更具可读性.
"""
question = "雷电天气下民航飞机的飞行风险及预防措施分析中, 研究目的是什么?"

filename = "out/mh_src.json"
loader = JSONLoader(
    file_path=filename,
    jq_schema='.[].answer',
    text_content=False
)
docs = loader.load()


In [60]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores.faiss import FAISS
embedding = HuggingFaceInstructEmbeddings(
    model_name="../models/stella-large-zh-v2"
)
db = FAISS.from_documents(docs, embedding)

retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 2, 'lambda_mult': 0.15})


No sentence-transformers model found with name ../models/stella-large-zh-v2. Creating a new one with MEAN pooling.


In [9]:
info = retriever.get_relevant_documents(query=question)
content='\n'.join([i.page_content for i in info])
print(len(content))
print(content)

NameError: name 'question' is not defined

In [8]:
prompt = PromptTemplate.from_template(_template)
from langchain.schema.output_parser import StrOutputParser
from query_data import get_llm
llm = get_llm('openai_3')
chain = prompt | llm | StrOutputParser()


In [52]:
out = chain.invoke({"context": content})
print(out)

雷电天气下民航飞机的飞行风险及预防措施的研究旨在深入分析雷电天气对民航飞机的影响，并提出有效的预防措施，以提高飞行安全水平。首先，通过研究雷电天气对飞机设备、飞行性能和导航系统等方面的损害，可以揭示雷电天气对飞机的物理规律和本质特征。其次，研究飞行员和地面人员在遇到雷电天气时的应对能力和心理承受能力，可以为相关培训和教育措施的制定提供科学依据。另外，研究气象预报和预警系统的准确性问题，可以提高预报和预警的准确性，及时向飞行员和地面人员发布警报。此外，研究针对雷电天气引发的飞行事故的应急预案和救援措施，可以提供有效的应对措施。最后，关注航空航天领域的技术研发和创新，例如飞行器防雷技术和天气预测技术，可以降低雷电天气对民航飞行的影响。通过对这些关键因素和问题的深入研究，我们可以为实际操作提供有效的建议和指导，提高民航飞行的安全性和可靠性。


In [7]:
from query_data import get_llm

llm = ChatOpenAI(
    openai_api_base="https://aiapi.xing-yun.cn/v1",
    openai_api_key="sk-3e5wTBAl2iFDvQvW9b5693C90a97425eBf3b4bEa558eC66a",
    temperature=0.25,
    model_name="gpt-3.5-turbo",
    # streaming=True,  # ! important
    # callbacks=[StreamingStdOutCallbackHandler()]  # ! important
)
temp = """
原文:
{text}

对原文扩写,只写一个段落, 作为论文关于雷电天气下民航飞机的飞行风险及预防措施分析的研究目的的一部份，并引用相关的具体研究和数据支持你的论点。"""
write_prompt = PromptTemplate.from_template(temp)
write_chain = write_prompt | llm | StrOutputParser()
rt = write_chain.invoke({"context": content, "text":out})
print(rt)


NameError: name 'content' is not defined